In [1]:
import pandas as pd
from modeling_old import clear_column_names, regressor

In [2]:
def load_data(path="./df_3ap_final.csv"):
    df = pd.read_csv(path, header=[0, 1], index_col=None)

    y = df[("seq_time", "_")]
    X = df.drop(columns=[("seq_time", "_")])

    X.columns = clear_column_names(X)

    return X, y

In [3]:
X, y = load_data()
X_test, y_test = load_data("./df_1_3ap_test_final.csv")
X.shape, X_test.shape

((691, 338), (105, 338))

In [4]:
y_test

0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
       ..
100   NaN
101   NaN
102   NaN
103   NaN
104   NaN
Name: (seq_time, _), Length: 105, dtype: float64

In [5]:
regressor(X, y, "xgboost")


Feature Importances:
                                          Feature  Importance
315         ap_from_ap_B_mean_rssi_le_nav_percent    0.167334
274         ap_from_ap_A_mean_rssi_le_nav_percent    0.052442
294       ap_from_ap_B_max_rssi_in_nav_ed_percent    0.038273
87         sta_to_ap_2_max_ant_rssi_la_ed_percent    0.032613
149              sta_from_ap_0_mean_ant_rssi_sinr    0.031316
..                                            ...         ...
169  sta_from_ap_1_max_ant_rssi_in_nav_ed_percent    0.000000
154             sta_from_ap_0_mean_ant_rssi_range    0.000000
155               sta_from_ap_0_mean_ant_rssi_iqr    0.000000
157               sta_from_ap_0_mean_ant_rssi_var    0.000000
175              sta_from_ap_1_max_ant_rssi_range    0.000000

[338 rows x 2 columns]
{'mse': {'train': [15.58654623864419, 14.971061124662903, 14.771109994318152, 15.350514169848278, 15.722746699172362], 'test': [45.94056037796103, 50.208811205308685, 61.12342064606214, 51.24134768803414, 42.86

{'shap_values':                                         Feature  SHAP Value  SHAP Sign
 315       ap_from_ap_B_mean_rssi_le_nav_percent    0.353566        1.0
 274       ap_from_ap_A_mean_rssi_le_nav_percent   -0.335449       -1.0
 86   sta_to_ap_2_max_ant_rssi_in_nav_ed_percent   -0.097607       -1.0
 302                  ap_from_ap_B_max_rssi_mean    0.061723        1.0
 149            sta_from_ap_0_mean_ant_rssi_sinr    0.059922        1.0
 ..                                          ...         ...        ...
 197             sta_from_ap_1_mean_ant_rssi_iqr    0.000000        0.0
 129           sta_from_ap_0_max_ant_rssi_length    0.000000        0.0
 151             sta_from_ap_0_mean_ant_rssi_max    0.000000        0.0
 150          sta_from_ap_0_mean_ant_rssi_length    0.000000        0.0
 236             sta_from_ap_2_mean_ant_rssi_min    0.000000        0.0
 
 [338 rows x 3 columns],
 'feature_importances':                                           Feature  Importance
 315    

In [6]:
import xgboost as xgb

model = xgb.XGBRegressor(
    objective="reg:squarederror",
    n_estimators=100,
    max_depth=3,
    learning_rate=0.1,
    random_state=42,
)

model.fit(X, y)
y_test_pred = model.predict(X_test)

In [7]:
# save predictions
y_test_pred = pd.Series(y_test_pred, name="seq_time")
y_test_pred.to_csv("./results/y_1_3ap_test_pred.csv", index=False)